In [18]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import os
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import re
import string
from nltk import ngrams
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
strDataDir = '/project/hackathon/hackers09/hack095/NOVEL-CLINICAL-PREDICTION-APPROACHES-TO-MANAGING-CARE-FOR-ACUTE-PULMONARY-EMBOLISM-PATIENTS/'
strMergedCsv = os.path.join(strDataDir, 'enc_outcome_echo.csv')

In [3]:
dfMerged = pd.read_csv(strMergedCsv, index_col=0)

In [12]:
def preproc_string(strInput):
    strInput = strInput.lower() # to lower case
    strInput = re.sub(r'\d+','',strInput) # remove numbers
    # remove punctuation
    exclude = set(string.punctuation)
    strInput = ''.join(ch for ch in strInput if ch not in exclude)
    # remove stop words
    exclude = ENGLISH_STOP_WORDS
    strInput = ''.join(ch for ch in strInput if ch not in exclude)

    return strInput

In [9]:
def get_max_fuzz(strInput, nGram, objFuzzFunction, strCompare):
    tokens = word_tokenize(strInput)
    lsThreegrams = list(ngrams(tokens, nGram))
    nMaxFuzz = 0
    for tupGram in lsThreegrams:
        strGram = ' '.join(tupGram)
        nTempFuzz = objFuzzFunction(strInput, strCompare)
        if nTempFuzz > nMaxFuzz:
            nMaxFuzz = nTempFuzz
            
    return nMaxFuzz

In [7]:
lsEchoCategories = ['no dilation', 'normal dilation', 'mild dilation', 
                    'min dilation', 'mod dilation', 'severe dilation',
                    'normal function', 'no function', 'mild function', 'min function', 
                    'mod function', 'severe function', 'mcconnell sig', 'lv function']

In [8]:
dfEchoFuzzy = pd.DataFrame(columns=['NARRATIVE_compiled']+lsEchoCategories,
                           index=dfMerged.index)
dfEchoFuzzy['NARRATIVE_compiled'] = dfMerged['NARRATIVE_compiled'].fillna('')

In [13]:
for nEnc in dfEchoFuzzy.index:
    strNarrative = dfEchoFuzzy.at[nEnc, 'NARRATIVE_compiled']
    strNarrative = preproc_string(strNarrative)
    for strCol in lsEchoCategories:            
        dfEchoFuzzy.at[nEnc, strCol] = get_max_fuzz(strNarrative, 3, fuzz.ratio, strCol)

In [28]:
dfEchoFuzzy.max()['NARRATIVE_compiled']

'· Unable to estimate right ventricular systolic pressure due to * insufficient tricuspid regurgitation Doppler signal.· LV function appears low normal to mildly decreased. Unable to quantify * LV EF despite use of contrast.· RV mildly dilated with intact basal function (TAPSE 1.9cm) and overally * low normal function.· Valves not well seen. No significant dysfunction by doppler.· The inferior vena cava was not visualized. * ***'

In [ ]:
dfEchoFuzzy['SevereSum'] = dfEchoFuzzy['']